In [1]:
import csv 
import json 
from collections import Counter
#import spacy
#nlp = spacy.load('en_core_web_sm', disable=['ner'])
from tqdm import tqdm
from spacy.matcher import Matcher
from spacy.tokens import Span 
import nltk
from nltk.corpus import stopwords
skip_words = set(stopwords.words('english'))
import stanza
nlp = stanza.Pipeline('en')

from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

do_lemmatize = True

2020-05-10 16:53:58 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-05-10 16:53:58 INFO: Use device: gpu
2020-05-10 16:53:58 INFO: Loading: tokenize
2020-05-10 16:54:00 INFO: Loading: pos
2020-05-10 16:54:01 INFO: Loading: lemma
2020-05-10 16:54:01 INFO: Loading: depparse
2020-05-10 16:54:02 INFO: Loading: ner
2020-05-10 16:54:02 INFO: Done loading processors!


In [2]:
import unicodedata
def normalize(text):
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return str(text)

def read_data(file):
    data = []
    with open(file, 'r') as f:
        for line in f:
            sample = line.strip().split('\t')
            assert len(sample) == 5
            data.append(sample)
    return data

In [3]:
def read_garbage(filelist, lead='verb'):
    garbage = {}
    concept_len = Counter()
    for file in filelist:
        with open(file, 'r') as f:
            reader = csv.reader(f, delimiter=',')
            for i, line in enumerate(reader):
                if do_lemmatize:
                    line[0] = lmtzr.lemmatize(line[0].lower())
                    line[1] = lmtzr.lemmatize(line[1].lower())
                    line[2] = lmtzr.lemmatize(line[2].lower())
                subj, verb, obj = line[0],line[1], line[2]
                if lead == 'verb':
                    head1, head2, head3 = verb, subj, obj
                elif lead == 'subj': 
                    head1, head2, head3 = subj, obj, verb
                else:
                    head1, head2, head3 = obj, subj, verb
                if head1 not in garbage:
                    garbage[head1] = {head2:{head3:[line[3]]}}
                elif head2 not in garbage[head1]:
                    garbage[head1][head2] = {head3:[line[3]]}
                elif head3 not in garbage[head1][head2]:
                    garbage[head1][head2][head3] = [line[3]]
                else:
                    garbage[head1][head2][head3] += [line[3]]
            print (i)
        print (len(garbage))
    return garbage
garbage_verb = read_garbage(['/home/kaixinm/kaixinm/Wikipedia/python-sutime/DURATION.csv',\
                             '/home/kaixinm/kaixinm/Wikipedia/python-sutime/DURATION_wiki_4_cnn_dai_gut.csv'])
garbage_subj = read_garbage(['/home/kaixinm/kaixinm/Wikipedia/python-sutime/DURATION.csv', \
                             '/home/kaixinm/kaixinm/Wikipedia/python-sutime/DURATION_wiki_4_cnn_dai_gut.csv'], lead='subj')
garbage_obj = read_garbage(['/home/kaixinm/kaixinm/Wikipedia/python-sutime/DURATION.csv', \
                            '/home/kaixinm/kaixinm/Wikipedia/python-sutime/DURATION_wiki_4_cnn_dai_gut.csv'], lead='obj')
# garbage_verb = read_garbage('./DURATION.csv')
# garbage_subj = read_garbage('./DURATION.csv', lead='subj')
# garbage_obj = read_garbage('./DURATION.csv', lead='obj')

5411527
49001
1748543
57710
5411527
269744
1748543
341907
5411527
101428
1748543
124667


In [114]:
print (garbage_verb['last']['eclipse'])

{'': ['3 hours', '18 minutes', '3 hours', '33 minutes', '3 hours', '36 minutes', '3 hours', '40 minutes', '3 hours', '39 minutes', 'a few minutes', '38 days', '32 days', 'up to a few minutes', '7\xa0minutes', '31\xa0seconds', '220 minutes', 'hours', 'about five hours', 'hours']}


In [4]:
def check_role(label):
    print (label)
    if 'subj' in label or 'obj' in label:
        return True
    else:
        return False
def extract_info(content):
    doc = nlp(content)
    query = [[], [], []]
    all_words = doc.sentences[0].words
    root = [w for w in doc.sentences[0].words if w.deprel == 'root'][0]
    if content.startswith('it'):
        print (content)
        #print (all_words)
    if not root.xpos.startswith('VB'):
        #print (all_words)
        if root.xpos.startswith('JJ'):
            #query[1].append(root.text)
            for w in doc.sentences[0].words:
                if 'root' in w.deprel:
                    query[1].append(w.text)
                if 'subj' in w.deprel:
                    query[0].append(w.text)
                if 'nmod' in w.deprel:
                    query[2].append(w.text)
            #print (content)
        else:
        #elif root.xpos.startswith('NN'):
            #query[0].append(root.text)
            for w in doc.sentences[0].words:
                if 'root' in w.deprel:
                    query[0].append(w.text)
                if 'amod' in w.deprel and len(query[0]) != 0:
                    query[1].append(w.text)
                if 'nmod' in w.deprel and len(query[0]) != 0:
                    query[2].append(w.text)
            #print (content)
        #else:
        #    print (all_words)
        #if 'last' in content:
        #    query[1].append('last')
    else:
        for w in  doc.sentences[0].words:

            if w.head == 0:
                query[1].append(w.text)
            if (all_words[w.head-1].head == 0 and 'subj' in w.deprel):
                query[0].append(w.text)
            if (all_words[w.head-1].head == 0 and 'obj' in w.deprel):
                query[2].append(w.text)
    query = [' '.join(part) for part in query]
    #print (query)
    return query

from collections import defaultdict
def find_patterns(data):
    # return: dict 
    # key: question str 
    # value: tokenized_query: list, level: int, raw_times: list
    start = Counter()
    prev = ''
    question_to_range_map = defaultdict(lambda: defaultdict(dict))
    # question_to_range_map = defaultdict()
    for sample in data:
        if sample[-1] == 'Event Duration':
            if prev == sample[1]:
                continue
            prev = sample[1]
            question = nltk.word_tokenize(sample[1].lower())
            q_tags = nltk.pos_tag(question)
            if do_lemmatize:
                question = [lmtzr.lemmatize(word) for word in question]

            if 'how' not in question:
                content = sample[1].lower()
            elif not q_tags[2][1].startswith('VB') and 'would' not in question:
                for i in range(3, len(q_tags)):
                    if q_tags[i][1].startswith('VB'):
                        break
                if len(question)>i+3 and question[i+1] == 'it' and (question[i+2] == 'take' or question[i+2] == 'taken'):
                    q_words = ' '.join(question[:i+3])
                    content = ' '.join(question[i+3:])
                else:
                    q_words = ' '.join(question[:i+1])
                    content = ' '.join(question[i+1:])
            else:
                if len(question)>5 and question[3] == 'it' and (question[4] == 'take' or question[4] == 'taken'):
                    q_words = ' '.join(question[:5])
                    content = ' '.join(question[5:])
                else:
                    q_words = ' '.join(question[:3])
                    content = ' '.join(question[3:])
            start[q_words] += 1
            if len(content) == 0:
                continue
            # ['minority', 'remained', '']
            query = extract_info(content)
            level, results = search_in_garbage(query, garbage_verb)
            if level == 2:
                subj_level, subj_results = search_in_garbage_subj(query, garbage_subj, garbage_obj)
                if subj_level == 2:
                    results += subj_results
            elif level == 1:
                subj_level, subj_results = search_in_garbage_subj(query, garbage_subj, garbage_obj)
                if subj_level == 2:
                    results = subj_results 
                    level = subj_level
                elif subj_level == 1:
                    results += subj_results
            elif level == 0:
                level, results = search_in_garbage_subj(query, garbage_subj, garbage_obj)
#             print (query)
#             print (level)
#             print (results)
#             break
            
            question_to_range_map[sample[1]]['tokenized_query'] = query
            question_to_range_map[sample[1]]['level'] = level
            question_to_range_map[sample[1]]['raw_times'] = results
            # start converting results to range
    #print (start)
    return question_to_range_map

In [5]:
def search_in_garbage(query, garbage):
#     import pdb; pdb.set_trace()
    if query[1] != '' and query[1] in garbage:
        if query[0] != '' and query[0] in garbage[query[1]]:
            if query[2] != '' and query[2] in garbage[query[1]][query[0]]:
                results = garbage[query[1]][query[0]][query[2]]      # all 3 words matched
                return 3, results   # how many words matched
            elif query[2] != '':   # there exists object but not matched 
                results = [vv for obj, v in garbage[query[1]][query[0]].items() for vv in v]  # stuff matched by verb and subj
                for subj, d in garbage[query[1]].items():  # search for all that could be matched by verb and obj
                    for obj, v in d.items():
                        if obj == query[2]:
                            results += v
                return 2, results     # all level 2 matched
            else:
                results = [vv for obj, v in garbage[query[1]][query[0]].items() for vv in v]  # only has verb and subj, already the best we can do
                return 2, results 
        elif query[2] != '':  # subj not matched and obj exist
            results = []
            for subj, d in garbage[query[1]].items():  # search for all that could be matched by verb and obj
                for obj, v in d.items():
                    if obj == query[2]:
                        results += v
            if len(results) > 0:
                return 2, results
            else:        # the best we can do is matching only with verb
                for subj, d in garbage[query[1]].items():
                    for obj, v in d.items():
                        results += v
                return 1, results
        else:   # the best we can do is matching only with verb
            results = []
            for subj, d in garbage[query[1]].items():
                for obj, v in d.items():
                    results += v
            return 1, results
    return 0, []

In [6]:
def search_in_garbage_subj(query, garbage, next_garbage):
    if query[0] != '' and query[0] in garbage:
        if query[2] != '' and query[2] in garbage[query[0]]:  # we search for subj and obj match
                results = [vv for obj, v in garbage[query[0]][query[2]].items() for vv in v]
                return 2, results 
        elif query[2] != '':  # obj not matched but exist
            results = []   # all matched with subj
            for obj, d in garbage[query[0]].items():
                for verb, v in d.items():
                    results += v
            results += search_in_garbage_obj(query, next_garbage)
            return 1, results
        else:   # the best we can do is matching only with subj
            results = []
            for obj, d in garbage[query[0]].items():
                for verb, v in d.items():
                    results += v
            return 1, results
    elif query[2] != '':
        results = []
        results += search_in_garbage_obj(query, next_garbage)
        return 1, results
    return 0, []
def search_in_garbage_obj(query, garbage):
    if query[2] != '' and query[2] in garbage:
        results = []
        for subj, d in garbage[query[2]].items():
            for verb, v in d.items():
                results += v
        return results
    return []

In [7]:
def expand_map_with_range(question_to_range_map):
    unit_keys = ['seconds', 'minutes', 'hours', 'days', 'weeks', 'months', 'years', 'decades', 'centuries']
    if do_lemmatize:
        unit_keys = [lmtzr.lemmatize(key) for key in unit_keys]
    for k, v in question_to_range_map.items():
        unit_count_map = defaultdict(lambda:0, {k: 0 for k in unit_keys})
        raw_times = [normalize(i.replace('-', ' ')) for i in v['raw_times']]
        for time in raw_times:
            time_tokenized = nltk.word_tokenize(time)
            if do_lemmatize:
                time_tokenized = [lmtzr.lemmatize(tok) for tok in time_tokenized]
            for tok in time_tokenized:
                if tok in unit_count_map:
                    unit_count_map[tok] += 1

        # sort unit_count_map desc
        if second_sort_by_desc_unit:
            unit_count_sorted = sorted(unit_count_map.items(), key=lambda v: (v[1], unit_keys.index(v[0])), reverse=True)
        else:
            unit_count_sorted = sorted(unit_count_map.items(), key=lambda v: v[1], reverse=True)
        question_to_range_map[k]['unit_count_sorted'] = unit_count_sorted
        question_to_range_map[k]['max_unit'] = unit_count_sorted[0]
        question_to_range_map[k]['min_unit'] = unit_count_sorted[1]
        # generate range
        if unit_keys.index(unit_count_sorted[0][0]) < \
            unit_keys.index(unit_count_sorted[1][0]):
            all_ranged_units = unit_keys[unit_keys.index(unit_count_sorted[0][0]):unit_keys.index(unit_count_sorted[1][0])+1]
        else:
            all_ranged_units = unit_keys[unit_keys.index(unit_count_sorted[1][0]):unit_keys.index(unit_count_sorted[0][0])+1]
            
        question_to_range_map[k]['range'] = all_ranged_units
    return question_to_range_map

In [8]:
def get_result_from_range(norm_data, range_map):
    results = []
    for sample in norm_data:
        if sample[-1] == 'Event Duration':
            ranged_units = range_map[sample[1]]['range']
            answer = nltk.word_tokenize(sample[2].lower())
            answer = [lmtzr.lemmatize(word) for word in answer]
            found = False
            for tok in answer:
                if tok in ranged_units:
                    found = True
                    break
            results.append('yes' if found else 'no')
    return results
            
    

In [16]:
do_lemmatize = True
second_sort_by_desc_unit = False
data = read_data('../../../MCTACO/dataset/norm_test_9442.tsv')
question_to_time_map = find_patterns(data)

it existed ?
it last ?
it a violent world


In [10]:
question_to_range_map = expand_map_with_range(question_to_time_map)
normalized_data = read_data('../../../MCTACO/dataset/norm_test_9442.tsv')
res = get_result_from_range(normalized_data, question_to_range_map)

In [18]:
with open('../../../MCTACO/dataset/garbage_devtuned_rule_result.txt', 'w') as f:
    for ans in res:
        f.write(ans+'\n')

In [33]:
import random
random.seed(15213)
def dump_baselines(res):
    with open('../../../MCTACO/dataset/random_result.txt', 'w') as f:
        for ans in res:
            if random.random() > 0.5:
                f.write('yes\n')
            else:
                f.write('no\n')
    with open('../../../MCTACO/dataset/positive_result.txt', 'w') as f:
        for ans in res:
            f.write('yes\n')
    with open('../../../MCTACO/dataset/negative_result.txt', 'w') as f:
        for ans in res:
            f.write('no\n')
dump_baselines(res)

In [ ]:
# Dev random
# Strict Acc.: 0.031746031746031744
# Avg F1: 0.3056352377780948
# Dev negative
# Strict Acc.: 0.24603174603174602
# Avg F1: 0.24603174603174602
# Dev positive
# Strict Acc.: 0.015873015873015872
# Avg F1: 0.3305273982573848
# Dev garbage 
# Strict Acc.: 0.19047619047619047
# Avg F1: 0.38031987389756794

In [ ]:
# Test positive 
# Strict Acc.: 0.022292993630573247
# Avg F1: 0.37325883850538083
# Test negative
# Strict Acc.: 0.2197452229299363
# Avg F1: 0.2197452229299363
# Test random 
# Strict Acc.: 0.028662420382165606
# Avg F1: 0.28177336796669894
# Test garbage 
# Strict Acc.: 0.15605095541401273
# Avg F1: 0.3741067949883198

In [50]:
# 1. lemmatize garbage subj/obj/verb
# 2. start generating ranges for each Q 

In [22]:
def write_data(filename, data):
    with open(filename, 'w') as fout:
        json.dump(data, fout)
write_data('dev_3783_garbage_part1.json', question_to_time_map)

In [10]:
from time_normalize import TimeNormalize
norm = TimeNormalize()
# convert question_to_range_map 
import statistics

def expand_map_with_seconds_stats(norm, question_to_range_map, add_quantization=False, drop_century=True):
    """expand dictionary and add fields 'converted_seconds', 'min', 'max', 'avg', 'std', 'quantization'(optional)
    norm(object)
    question_to_range_map(dict) 
    """
    if add_quantization:
        raise NotImplementedError
    for k, v in question_to_range_map.items():
        question_to_range_map[k]['converted_seconds'] = []
        for t in v['raw_times']:
            if drop_century and ('century' in t or 'centuries' in t):
                continue
            success, second = norm.convert_time_to_seconds(t)
#             if success and second > 100:
#                 print (t, second)
#                 break
            if success:
                question_to_range_map[k]['converted_seconds'].append(second)
        
        # get stats
        try:
            question_to_range_map[k]['min'] = min(question_to_range_map[k]['converted_seconds'])
            question_to_range_map[k]['max'] = max(question_to_range_map[k]['converted_seconds'])
            question_to_range_map[k]['avg'] = statistics.mean(question_to_range_map[k]['converted_seconds'])
            question_to_range_map[k]['std'] = statistics.stdev(question_to_range_map[k]['converted_seconds'])
        except ValueError:
            if len(question_to_range_map[k]['converted_seconds']) == 1:
                question_to_range_map[k]['std'] = 0.0
            elif len(question_to_range_map[k]['converted_seconds']) == 0:
                question_to_range_map[k]['min'] = 0.0
                question_to_range_map[k]['max'] = 0.0
                question_to_range_map[k]['avg'] = 0.0
                question_to_range_map[k]['std'] = 0.0
            else:
                print(k)
                print(question_to_range_map[k])
#         if add_quantization:
            # quantize by ['seconds', 'minutes', 'hours', 'days', 'weeks', 'months', 'years', 'decades', 'centuries']
            
            
    return question_to_range_map

In [11]:
def compute_distance(cand, group):
    distance = 0.0
    for t in group:
        distance += abs(cand-t)
    if len(group) == 0:
        return 0.0
    #print (distance, len(group))
    distance /= len(group)
    return distance

def normalize_dist(dist):
    if len(dist) == 0:
        return dist
    avg = sum(dist)/len(dist)
    dist = [d-avg for d in dist]
    return dist

def get_result_from_seconds_min_max(data, range_map):
    unit_keys = ['seconds', 'minutes', 'hours', 'days', 'weeks', 'months', 'years', 'decades', 'centuries']
    unit_keys = [lmtzr.lemmatize(key) for key in unit_keys]
    results = []
    count = 0
    all_dist = []
    all_labels = []
    tmp_dist = []
    prev = ''
    tmp_label = []
    for sample in data:
        if sample[-1] == 'Event Duration':
            min_second = range_map[sample[1]]['min']
            max_second = range_map[sample[1]]['max']
            try:
                answer = float(norm.convert_time_to_seconds(sample[2])[1])
            except:
                words = nltk.word_tokenize(sample[2].lower())
                lemmas = [lmtzr.lemmatize(word) for word in words]
                ans_str = ''
                for i, w in enumerate(lemmas):
                    if w in unit_keys:
                        ans_str = ' '.join(words[i-1:i+1])
                        break
                try:
                    answer = float(norm.convert_time_to_seconds(ans_str)[1])
                except:
                    answer = 1.0
            #answer = float(sample[2])
            if sample[1] != prev:
                tmp_dist = normalize_dist(tmp_dist)
                print (tmp_dist)
                print (tmp_label)
                all_dist += tmp_dist
                tmp_dist = []
                tmp_label = []
                #break
            dist = compute_distance(answer, range_map[sample[1]]['converted_seconds'])
#             if dist > 100:
#                 print (dist)
#                 print (answer, range_map[sample[1]]['converted_seconds'])
#                 break
            #all_dist.append(dist)
            tmp_dist.append(dist)
            tmp_label.append(sample[3])
            all_labels.append(sample[3])
#             if min_second <= answer <= max_second:
#                 results.append('yes')        
#             else:
#                 results.append('no')
            count += 1
#         if count == 20:
#             break
            prev = sample[1]
    f1 = compute_f1(all_dist, all_labels)
    #print (all_dist)
    #print (all_labels[:50])
    print (f1)
    tmp_dist = []
    prev = ''
    for sample in data:
        if sample[-1] == 'Event Duration':
            min_second = range_map[sample[1]]['min']
            max_second = range_map[sample[1]]['max']
            try:
                answer = float(norm.convert_time_to_seconds(sample[2])[1])
            except:
                words = nltk.word_tokenize(sample[2].lower())
                lemmas = [lmtzr.lemmatize(word) for word in words]
                ans_str = ''
                for i, w in enumerate(lemmas):
                    if w in unit_keys:
                        ans_str = ' '.join(words[i-1:i+1])
                        break
                try:
                    answer = float(norm.convert_time_to_seconds(ans_str)[1])
                except:
                    answer = 1.0
            #answer = float(sample[2])
            dist = compute_distance(answer, range_map[sample[1]]['converted_seconds'])
            if sample[1] != prev:
                tmp_dist = normalize_dist(tmp_dist)
                #print (tmp_dist)
                if all([d==0 for d in tmp_dist]):
                    results += ['no' for _ in tmp_dist]
                else:
                    for dd in tmp_dist:
                        if dd < 2.58:
                            results.append('yes')
                        else:
                            results.append('no')
                tmp_dist = []
            tmp_dist.append(dist)
            prev = sample[1]
    if all([d==0 for d in tmp_dist]):
        results += ['no' for _ in tmp_dist]
    else:
        for dd in tmp_dist:
            if dd < 2.58:
                results.append('yes')
            else:
                results.append('no')
    print (results[:50])
    print (len(results))
    return results

In [17]:
question_to_range_map = expand_map_with_seconds_stats(norm, question_to_time_map, add_quantization=False, drop_century=True)
res = get_result_from_seconds_min_max(data, question_to_range_map)

[]
[]
[15.92307692307692, -3.07692307692308, 60.92307692307692, -29.07692307692308, 9.92307692307692, 0.9230769230769198, -29.07692307692308, -27.07692307692308, 10.42307692307692, -3.07692307692308, 10.92307692307692, -6.07692307692308, -11.57692307692308]
['no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no']
[-6.475514069264364, -1.3669351481835292, 1.7570866633368496, 1.7570866633368496, -3.7149413086910954, -4.886269980019733, -3.1829316516830204, -1.5133512320996942, -1.9416729104241313, 19.56744297369346]
['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no']
[-9.303544888888887, -9.592433777777764, 11.05544939682538, 4.179947174603178, 3.660582095238091]
['yes', 'yes', 'yes', 'no', 'no']
[-1.5, 1.5]
['yes', 'no']
[614.2108378870689, -112.64604475669694, -93.85369502992398, -147.53935727296266, -34.5104735883433, -131.1003773093944, -93.02946916469227, -147.94919333853613, 74.0833333333212, 595.5223132969077, -139.57032266458555, -122.53675513921

['yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no']
[-8.36988890171051, 12.844539403915405, 4.481445014476776, 5.416925609111786, -9.334369510412216, -3.2813313603401184, -5.953905016183853, 4.481445014476776, -7.2384578585624695, 2.4721524715423584, 4.481445014476776]
['yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no']
[-13.259887727448717, 17.23757452574516, 16.95265582655835, 14.448590785907847, -0.9615718157181163, -6.967495160665898, -3.562588075880754, -1.4843360433604431, -14.849608981804117, -1.1794579945799555, -13.721560201316311, -1.4843360433604431, 17.10915989159883, 16.95265582655835, -0.8900271002709914, -10.607448703058488, -9.300828493999195, -4.431490514905139]
['no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no']
[0.0]
['yes']
[-0.18506447382276292, -20.675817687366475, -18.664287743760497, 17.783637246915077, 4.12237785743298, 17.619154800601677]
['no', 'yes', 'yes', '

[3.7796333828009665, -2.086022369330749, -7.029009624850005, 12.359549797140062, -6.905806161463261, 16.860424428014085, -6.710936319082975, 0.06524113705381751, -4.731788691831753, -4.154996666824445, -5.238238818245009, -7.0996174812316895, 17.797373553737998, -6.905806161463261]
['yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no']
[-1.6067209410785779, -1.2108542534962226, -5.524032269948165, -2.0123349210975476, -1.0033122663544418, -1.0033122663544418, -1.9789544993953427, 14.430487241818867, -1.6523443387713854, -5.767583779391259, 2.2427637028751093, 13.31712670188169, -2.0123349210975476, -6.218593189590749]
['no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no']
[-31.60817810460867, -3.692494208313292, -31.60817810460867, -40.35368764614395, -43.332249196428165, -31.450916691675957, -50.162025921476015, -50.88164654857974, -40.78397810261231, 428.49221237142046, -42.10403342974314, -22.4557425781677, -40.059081

['no', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'yes']
3032


In [13]:
import numpy as np
def compute_f1(out, labels):
    best_thres = np.inf
    best_f1 = 0.0
    num_correct = 0
    den = len([_ for l in labels if l == 'yes'])
    instances = sorted(zip(out, labels)) #[(-np.inf, 0)] + 
    for i in range(len(instances)):
            if instances[i][1] == 'yes':
                    num_correct += 1
            den += 1
            if instances[i][0] > instances[i-1][0]:
                    f1 = 2.0*num_correct / den
                    if f1 > best_f1:
                            best_thres = (instances[i][0] + instances[i-1][0]) / 2
                            best_f1 = f1
    return {'threshold':best_thres, 'f1':best_f1}

In [58]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
def get_result_from_kmeans(data, range_map):
    unit_keys = ['seconds', 'minutes', 'hours', 'days', 'weeks', 'months', 'years', 'decades', 'centuries']
    unit_keys = [lmtzr.lemmatize(key) for key in unit_keys]
    results = []
    count = 0
    all_dist = []
    all_labels = []
    tmp_dist = []
    prev = ''
    tmp_label = []
    for sample in data:
        if sample[-1] == 'Event Duration':
            min_second = range_map[sample[1]]['min']
            max_second = range_map[sample[1]]['max']
            try:
                answer = float(norm.convert_time_to_seconds(sample[2])[1])
            except:
                words = nltk.word_tokenize(sample[2].lower())
                lemmas = [lmtzr.lemmatize(word) for word in words]
                ans_str = ''
                for i, w in enumerate(lemmas):
                    if w in unit_keys:
                        ans_str = ' '.join(words[i-1:i+1])
                        break
                try:
                    answer = float(norm.convert_time_to_seconds(ans_str)[1])
                except:
                    answer = 1.0
            #answer = float(sample[2])
            if sample[1] != prev and prev != '':
                tmp_dist = normalize_dist(tmp_dist)
                if all([d==0 for d in tmp_dist]):
                    results += ['no' for _ in tmp_dist]
                else:
                    x = np.reshape(np.array(tmp_dist), (len(tmp_dist),1))
                    if len(tmp_dist) > 2:
                        sil_scores = []
                        for k in range(2, len(tmp_dist)):
                            kmeans = KMeans(n_clusters=k, random_state=0).fit(x)
                            sil_scores.append(silhouette_score(x, kmeans.labels_, metric = 'euclidean'))
                        best_k = sil_scores.index(max(sil_scores))+2
                    else:
                        best_k = 2
                    kmeans = KMeans(n_clusters=best_k, random_state=0).fit(x)
                    #print (kmeans.labels_)
                    #print (tmp_dist)
                    print (tmp_label)
                    #print (kmeans.inertia_)
                    #print (kmeans.cluster_centers_)
                    this_res = []
                    centers = kmeans.cluster_centers_[:,0].tolist()
                    min_center = centers.index(min(centers))
                    for l in kmeans.labels_:
                        if l == min_center:
                            results.append('yes')
                            this_res.append('yes')
                        else:
                            results.append('no')
                            this_res.append('no')
                    print (this_res)
                #print (res)
                all_dist += tmp_dist
                tmp_dist = []
                tmp_label = []
                
                #break
            dist = compute_distance(answer, range_map[sample[1]]['converted_seconds'])
#             if dist > 100:
#                 print (dist)
#                 print (answer, range_map[sample[1]]['converted_seconds'])
#                 break
            #all_dist.append(dist)
            tmp_dist.append(dist)
            tmp_label.append(sample[3])
            all_labels.append(sample[3])
#             if min_second <= answer <= max_second:
#                 results.append('yes')        
#             else:
#                 results.append('no')
            count += 1
#         if count == 20:
#             break
            prev = sample[1]
    
    if all([d==0 for d in tmp_dist]):
        results += ['no' for _ in tmp_dist]
    else:
        x = np.reshape(np.array(tmp_dist), (len(tmp_dist),1))
        if len(tmp_dist) > 2:
            sil_scores = []
            for k in range(2, len(tmp_dist)):
                kmeans = KMeans(n_clusters=k, random_state=0).fit(x)
                sil_scores.append(silhouette_score(x, kmeans.labels_, metric = 'euclidean'))
            best_k = sil_scores.index(max(sil_scores))+2
        else:
            best_k = 2
        kmeans = KMeans(n_clusters=best_k, random_state=0).fit(x)
        centers = kmeans.cluster_centers_[:,0].tolist()
        min_center = centers.index(min(centers))
        for l in kmeans.labels_:
            if l == min_center:
                results.append('yes')
            else:
                results.append('no')
    print (results[:50])
    print (len(results))
    return results
res = get_result_from_kmeans(data, question_to_range_map)

['no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no']
['yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']


/hdd3/kaixinm/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:42: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
/hdd3/kaixinm/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:42: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (11). Possibly due to duplicate points in X.
/hdd3/kaixinm/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:42: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (12). Possibly due to duplicate points in X.
/hdd3/kaixinm/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:42: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (13). Possibly due to duplicate points in X.


['no', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes']
['no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes']
['no', 'no', 'no', 'no', 'no', 'no', 'no']
['no', 'no', 'yes', 'yes', 'no', 'yes', 'no']
['yes', 'yes', 'no']
['yes', 'yes', 'no']
['no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no']
['yes', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes']
['yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes']
['no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no']
['yes', 'no', 'no', 'no']
['yes', 'yes', 'no', 'yes']
['no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no']
['no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes']
['no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no']
['no', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'yes']


/hdd3/kaixinm/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:42: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.


['yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes']
['yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'yes']


...

['yes', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no']
['yes', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no']
['no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no']
['no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no']
['no', 'no', 'no']
['no', 'yes', 'no']
['no', 'no', 'no']
['no', 'yes', 'no']
['no', 'no', 'yes', 'yes', 'no', 'no', 'no']
['no', 'yes', 'yes', 'yes', 'no', 'no', 'yes']


/hdd3/kaixinm/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:42: ConvergenceWarning: Number of distinct clusters (12) found smaller than n_clusters (13). Possibly due to duplicate points in X.


['no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'yes']
['no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'yes']
['no', 'no', 'no']
['no', 'yes', 'no']
['no', 'no', 'no', 'no', 'no', 'no', 'no']
['yes', 'yes', 'no', 'no', 'yes', 'no', 'no']
['yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no']
['yes', 'no', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'yes']


/hdd3/kaixinm/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:42: ConvergenceWarning: Number of distinct clusters (10) found smaller than n_clusters (11). Possibly due to duplicate points in X.


['no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no']
['no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no']
['yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'no']
1112


In [120]:
def sep_data(file, pred_file):
    data = []
    with open(file, 'r') as f:
        for line in f:
            sample = line.strip().split('\t')
            assert len(sample) == 5
            data.append(sample)
    preds = []
    with open(pred_file, 'r') as f:
        for line in f:
            preds.append(line.strip())
    duration_preds = []
    for i, sample in enumerate(data):
        if sample[-1] == 'Event Duration':
            duration_preds.append(preds[i])
    print (len(duration_preds))
    with open('../../../MCTACO/dataset/bert_test_duration.tsv', 'w') as fout:
        for pred in duration_preds:
            fout.write(pred+'\n')
sep_data('../../../MCTACO/dataset/norm_test_9442.tsv', '../../exp/mctaco_base/eval_outputs.txt')

3032


In [25]:
def read_prediction(file):
    pred = []
    with open(file, 'r') as f:
        for line in f:
            pred.append(line.strip())
    return pred
def ensemble():
    pred1 = read_prediction('../../../MCTACO/dataset/best_nsp_test.txt')
    pred2 = read_prediction('../../../MCTACO/dataset/garbage_rule_result.txt')
    #pred3 = read_prediction('../../../MCTACO/dataset/garbage_rule_result1.txt')
    res = []
    for i, p in enumerate(pred1):
        vote = [p, pred2[i]]
        if all([v == 'yes' for v in vote]):
            res.append('yes')
        else:
            res.append('no')
        #majority = Counter(vote).most_common()[0][0]
        #res.append(majority)
    with open('../../../MCTACO/dataset/ensemble.txt', 'w') as f:
        for ans in res:
            f.write(ans+'\n')
ensemble()